In [ ]:
!pip install lenskit
!pip install tbb
!pip install matplotlib
!pip install seaborn

In [ ]:
from lenskit.datasets import ML100K
from lenskit import batch, topn, util
from lenskit import crossfold
from lenskit.algorithms import Recommender, als, item_knn, bias, user_knn, funksvd,Predictor
from lenskit import topn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
ml100k = ML100K('ml-100k')
ratings = ml100k.ratings

In [ ]:
# All algorithms used
algo_ii = item_knn.ItemItem(20)
algo_uu = user_knn.UserUser(20)
algo_bias = bias.Bias(damping=5)
algo_biasedMF = als.BiasedMF(50)
algo_ImplicitMF = als.ImplicitMF(50)
algo_funksvd = funksvd.FunkSVD(50)

In [ ]:
def eval_batch(name, algo, train, test):
    # before fitting the algorithm, we clone it. Some algorithms misbehave when fit multiple times.
    fittable = util.clone(algo)
    # our algorithms do not necessarily implement the Recommender interface, so we adapt them. 
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = name
    return recs

In [ ]:
all_recs = []
test_data = []
for train, test in crossfold.partition_users(ratings[['user', 'item', 'rating']], 5, crossfold.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval_batch('ItemItem', algo_ii, train, test))
    all_recs.append(eval_batch('UserUser', algo_uu, train, test))
    all_recs.append(eval_batch('Bias', algo_bias, train, test))
    all_recs.append(eval_batch('Bias MF', algo_biasedMF, train, test))
    all_recs.append(eval_batch('Implicit MF', algo_ImplicitMF, train, test))
    all_recs.append(eval_batch('Funksvd', algo_funksvd, train, test))

In [ ]:
all_recs = pd.concat(all_recs, ignore_index=True)

In [ ]:
test_data = pd.concat(test_data, ignore_index=True)

In [ ]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test_data)
results.head()

In [ ]:
results = results.reset_index()

In [ ]:

plt.rcParams.update({'font.size': 10})
chart = sns.barplot(x="Algorithm", hue="Algorithm", y="ndcg", data=results)
chart.set_xticklabels(chart.get_xticklabels(), rotation=90, horizontalalignment='right')
plt.legend()
plt.show()

In [ ]:
def eval_rating(name, algo):
    # before fitting the algorithm, we clone it. Some algorithms misbehave when fit multiple times.
    fittable = util.clone(algo)

    train = ratings[['user', 'item', 'rating']]
    fittable =fittable.fit(train)
    
    result = fittable.predict_for_user(196,(302,))
    result['Algorithm'] = name
    return result

In [ ]:
df = pd.DataFrame()
for i,v in {"ItemItem": algo_ii,"Bias":algo_bias,"UserUser":algo_uu,"BiasedMF":algo_biasedMF,"ImplicitMF":algo_ImplicitMF,"Funksvd":algo_funksvd}.items():
    df = df.append(eval_rating(i, v),ignore_index=True)
print(df.to_latex())

In [ ]:
df = pd.DataFrame()
for i,v in {"ItemItem": algo_ii,"Bias":algo_bias,"UserUser":algo_uu,"BiasedMF":algo_biasedMF,"ImplicitMF":algo_ImplicitMF,"Funksvd":algo_funksvd}.items():
    d = eval_rating(i, v)[302]
    d1 = eval_rating(i, v)[302]
    d2 = eval_rating(i, v)[302]
    df = df.append(pd.DataFrame({"algorithm":[i],"302 first run":[d],"302 second run":[d],"302 third run":[d2]}))

In [ ]:
print(df.to_latex())